<a href="https://colab.research.google.com/github/x1001000/Colab-Notebooks/blob/main/TF_Lite_models_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download TF Lite models

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1A0CuL6UerrbopTtDeHZWZmWnc8n4eC8c/view'
!gdown --fuzzy 'https://drive.google.com/file/d/1A5YaEutUyJogCMBRGI5S4F3NbU0zVP6Q/view'

Downloading...
From: https://drive.google.com/uc?id=1A0CuL6UerrbopTtDeHZWZmWnc8n4eC8c
To: /content/model_age_nonq.tflite
100% 4.28M/4.28M [00:00<00:00, 27.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A5YaEutUyJogCMBRGI5S4F3NbU0zVP6Q
To: /content/model_age_q.tflite
100% 2.14M/2.14M [00:00<00:00, 15.9MB/s]


# Download UTKFace dataset

In [ ]:
!kaggle datasets download -d jangedoo/utkface-new
!unzip utkface-new.zip -d dataset > unzip.log

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 330M/331M [00:08<00:00, 25.4MB/s]
100% 331M/331M [00:08<00:00, 41.4MB/s]


# Select model to run inference

In [ ]:
Select = "model_age_nonq.tflite" # @param ["model_age_nonq.tflite", "model_age_q.tflite"]

import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=Select)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

import glob
jpgs = glob.glob('dataset/UTKFace/*.jpg')
print(f'Running inference on {len(jpgs)} files/faces in UTKFace dataset...')
print()

import time
begin = time.time()
y_true, y_pred = [], []
for jpg in jpgs:
    age, gender = map(int, jpg.split('/')[-1].split('_')[:2])
    y_true.append(age)

    # im = Image.open(jpg).resize(input_shape[1:3])
    # image = np.asarray(im).astype(input_dtype) / 255
    raw = tf.io.read_file(jpg)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.image.resize(image, input_shape[1:3]) / 255

    # input_data = image.reshape(input_shape)
    input_data = tf.reshape(image, input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    estimated_age = output_data[0][0] * 116
    y_pred.append(estimated_age)

elapsed = time.time() - begin
print(f'Inference time: {elapsed:.1f} seconds')
print(f'Inference speed: {len(jpgs)/elapsed:.1f} FPS')
print()

MAE = tf.keras.losses.MAE(y_true, y_pred).numpy()
print(f'MAE: {MAE:.3f} yrs')
RMSE = tf.keras.losses.MSE(y_true, y_pred).numpy() ** 0.5
print(f'RMSE: {RMSE:.3f} yrs')

Running inference on 23708 files/faces in UTKFace dataset...

Inference time: 222.2 seconds
Inference speed: 106.7 FPS

MAE: 2.762 yrs
RMSE: 4.259 yrs


# Select model to run inference

In [ ]:
Select = "model_age_q.tflite" # @param ["model_age_nonq.tflite", "model_age_q.tflite"]

import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=Select)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

import glob
jpgs = glob.glob('dataset/UTKFace/*.jpg')
print(f'Running inference on {len(jpgs)} files/faces in UTKFace dataset...')
print()

import time
begin = time.time()
y_true, y_pred = [], []
for jpg in jpgs:
    age, gender = map(int, jpg.split('/')[-1].split('_')[:2])
    y_true.append(age)

    # im = Image.open(jpg).resize(input_shape[1:3])
    # image = np.asarray(im).astype(input_dtype) / 255
    raw = tf.io.read_file(jpg)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.image.resize(image, input_shape[1:3]) / 255

    # input_data = image.reshape(input_shape)
    input_data = tf.reshape(image, input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    estimated_age = output_data[0][0] * 116
    y_pred.append(estimated_age)

elapsed = time.time() - begin
print(f'Inference time: {elapsed:.1f} seconds')
print(f'Inference speed: {len(jpgs)/elapsed:.1f} FPS')
print()

MAE = tf.keras.losses.MAE(y_true, y_pred).numpy()
print(f'MAE: {MAE:.3f} yrs')
RMSE = tf.keras.losses.MSE(y_true, y_pred).numpy() ** 0.5
print(f'RMSE: {RMSE:.3f} yrs')

Running inference on 23708 files/faces in UTKFace dataset...

Inference time: 211.6 seconds
Inference speed: 112.1 FPS

MAE: 2.763 yrs
RMSE: 4.260 yrs


# GPU spec for reference

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d2dbf6f4-b188-d558-91bc-646ee4d414a6)
